# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [4]:
# Load the saved features from the .npz file
data = np.load('ec_merged_features.npz') 

# Access the concatenated features from the .npz file
X_merged_features = data['X_merged_features']

# Close the loaded file
data.close()

#print the merged array
X_merged_features

array([[[-1.90864583e-06, -1.01319508e-06, -1.25947210e-06, ...,
          3.05983596e-11,  5.69918532e-01,  1.43285998e+00],
        [-4.16825421e-05, -4.20260902e-05, -4.24717352e-05, ...,
          1.80793891e-11,  3.16028756e-01,  2.11285116e+00],
        [-1.90472102e-05, -1.92407524e-05, -1.70617167e-05, ...,
          2.03574174e-11,  3.68552716e-01,  1.69556851e+00],
        ...,
        [ 6.94002021e-06,  6.58408463e-06,  6.57031824e-06, ...,
          2.44995562e-11,  3.34763239e-01,  2.11152035e+00],
        [ 2.41499888e-05,  2.47479670e-05,  2.44680713e-05, ...,
          2.61209914e-11,  3.37369781e-01,  1.97140856e+00],
        [-1.60015896e-06, -3.41169756e-06, -2.49139569e-06, ...,
          7.24925208e-11,  4.56258399e-01,  1.62313244e+00]],

       [[ 4.44703238e-05,  4.35599754e-05,  4.57796671e-05, ...,
          1.79259348e-10,  2.21715280e-01,  3.09942440e+00],
        [ 5.61311547e-06,  4.87111596e-06,  6.55256693e-06, ...,
          3.87428576e-11,  2.51679355e

In [5]:
data = np.load('eyes_close_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [6]:
data = np.load('eyes_close_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [7]:
Y=Y
groups_array = np.hstack(group)

In [8]:
# Check the shape of the concatenated arrays
print(X_merged_features.shape, Y.shape, groups_array.shape)

(17325, 20, 238) (17325,) (17325,)


In [9]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [10]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [12]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_merged_features, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_merged_features[train_index], X_merged_features[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
61/61 [==============================] - 12s 53ms/step - loss: 0.3406 - accuracy: 0.8551 - val_loss: 0.3132 - val_accuracy: 0.8609
Epoch 2/200
61/61 [==============================] - 1s 24ms/step - loss: 0.1971 - accuracy: 0.9219 - val_loss: 0.1977 - val_accuracy: 0.9175
Epoch 3/200
61/61 [==============================] - 1s 23ms/step - loss: 0.1627 - accuracy: 0.9330 - val_loss: 0.1671 - val_accuracy: 0.9313
Epoch 4/200
61/61 [==============================] - 1s 23ms/step - loss: 0.1291 - accuracy: 0.9473 - val_loss: 0.1658 - val_accuracy: 0.9348
Epoch 5/200
61/61 [==============================] - 1s 22ms/step - loss: 0.1044 - accuracy: 0.9590 - val_loss: 0.1399 - val_accuracy: 0.9475
Epoch 6/200
61/61 [==============================] - 1s 22ms/step - loss: 0.0974 - accuracy: 0.9608 - val_loss: 0.1510 - val_accuracy: 0.9504
Epoch 7/200
61/61 [==============================] - 1s 20ms/step - loss: 0.0814 - accuracy: 0.9681 - val_loss: 0.1467 - val_accuracy: 0.944

In [13]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [14]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9775,0.9740,0.9818,0.9779,0.9774,0.9731,0.9818,0.9550,0.9550
1,0.9786,0.9850,0.9727,0.9788,0.9787,0.9848,0.9727,0.9573,0.9574
2,0.9665,0.9691,0.9647,0.9669,0.9666,0.9684,0.9647,0.9331,0.9331
3,0.9758,0.9761,0.9761,0.9761,0.9758,0.9754,0.9761,0.9515,0.9515
4,0.9775,0.9709,0.9852,0.9780,0.9774,0.9696,0.9852,0.9550,0.9551
5,0.9781,0.9762,0.9807,0.9784,0.9780,0.9754,0.9807,0.9561,0.9561
6,0.9688,0.9769,0.9613,0.9690,0.9689,0.9766,0.9613,0.9376,0.9378
7,0.9694,0.9672,0.9727,0.9699,0.9693,0.9660,0.9727,0.9388,0.9388
8,0.9763,0.9783,0.9750,0.9766,0.9763,0.9777,0.9750,0.9526,0.9527
9,0.9798,0.9873,0.9727,0.9799,0.9799,0.9871,0.9727,0.9596,0.9597


In [15]:
metrics_df.round(4).to_csv('COMBINED_EC_CNN_LSTM.csv', index = False)

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
result = pd.read_csv("COMBINED_EC_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9775,0.9740,0.9818,0.9779,0.9774,0.9731,0.9818,0.9550,0.9550
1,0.9786,0.9850,0.9727,0.9788,0.9787,0.9848,0.9727,0.9573,0.9574
2,0.9665,0.9691,0.9647,0.9669,0.9666,0.9684,0.9647,0.9331,0.9331
3,0.9758,0.9761,0.9761,0.9761,0.9758,0.9754,0.9761,0.9515,0.9515
4,0.9775,0.9709,0.9852,0.9780,0.9774,0.9696,0.9852,0.9550,0.9551
5,0.9781,0.9762,0.9807,0.9784,0.9780,0.9754,0.9807,0.9561,0.9561
6,0.9688,0.9769,0.9613,0.9690,0.9689,0.9766,0.9613,0.9376,0.9378
7,0.9694,0.9672,0.9727,0.9699,0.9693,0.9660,0.9727,0.9388,0.9388
8,0.9763,0.9783,0.9750,0.9766,0.9763,0.9777,0.9750,0.9526,0.9527
9,0.9798,0.9873,0.9727,0.9799,0.9799,0.9871,0.9727,0.9596,0.9597


In [18]:
(result.mean()*100).round(2)

Accuracy       97.48
Precision      97.61
Recall         97.43
F1             97.52
Sensitivity    97.48
Specificity    97.54
ROC_AUC        97.43
Kappa          94.97
MCC            94.97
dtype: float64

In [19]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])